### 목록
* 준비
    * [사용할 라이브러리 불러오기](#lib)
    * [엘라스틱서치 환경 설정하기](#es_setting)
    * [엘라스틱서치에서 데이터 조회하기](#es_load)
    * [조회한 데이터를 테이블 형태로 만들기](#df)
* Sibling Pipeline Aggregation
    * [Sum Bucket Aggregation](#sum)
    * [Average Bucket Aggregation](#avg)
    * [Min Bucket Aggregation](#min)
    * [Max Bucket](#max)
---

### 준비

<a name='lib'></a>
#### 사용할 라이브러리 불러오기

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import datetime

<a name='es_setting'></a>
#### 엘라스틱서치 환경 설정하기

In [2]:
# 적당한 값을 넣지 않으면 오류가 납니다
es = Elasticsearch()
es_index = 'test'
es_type = 'test'

<a name='es_load'></a>
#### 엘라스틱서치에서 데이터 조회하기

In [3]:
x = es.search(
      index='test', 
      doc_type='test', 
      body={
        '_source' : ['주문시간', '상품가격', '판매자평점'],
        'query' : {
            'match_all' : {}
        },
        'size' : 100
      }
)

<a name='df'></a>
#### 조회한 데이터를 테이블 형태로 만들기

In [4]:
df = pd.DataFrame([x['hits']['hits'][idx]['_source'] for idx, _ in enumerate(x['hits']['hits'])])
df

,상품가격,주문시간,판매자평점
0,6000,2017-11-04T04:39:24,4
1,16000,2017-11-07T02:27:49,1
2,13000,2017-11-01T21:10:05,3
3,25000,2017-11-12T11:30:19,1
4,10000,2017-11-12T18:11:58,4
5,25000,2017-11-08T22:29:44,3
6,13000,2017-11-07T01:22:52,2
7,15000,2017-11-11T06:25:42,3
8,9000,2017-11-09T01:14:04,3
9,5000,2017-11-02T20:24:25,3


### Parent Piepline Aggregation

In [5]:
df['날짜'] = pd.to_datetime(df['주문시간'])
df['날짜'] = df['날짜'].apply(lambda x: '월'.join([str(x.month), str(x.day).zfill(2)]) + '일')
df['평점군'] = df['판매자평점'].apply(lambda x: '저평점' if 0<=x<=3 else '고평점')

<a name="sum"></a>
#### Sum Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 Sum한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 평균을 구하고 (Metric Aggregation : Average)
    * 그 평균(=상품가격들의 평균들의)들의 합을 구하시오

In [6]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).mean()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월01일,저평점,15200
1,11월02일,고평점,16000
2,11월02일,저평점,12000
3,11월03일,고평점,25000
4,11월03일,저평점,13000
5,11월04일,고평점,5500
6,11월04일,저평점,7000
7,11월05일,저평점,11000
8,11월06일,저평점,11666
9,11월07일,고평점,12500


In [7]:
# ... 합을 구하시오
df_gb_rate.groupby(['날짜']).sum().astype(int)

,상품가격
날짜,
11월01일,15200
11월02일,28000
11월03일,38000
11월04일,12500
11월05일,11000
11월06일,11666
11월07일,30500
11월08일,17000
11월09일,40333


<a name="avg"></a>
#### Average Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 Average 한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 합을 구하고 (Metric Aggregation : Sum)
    * 그 합(=상품가격들의 합들의)들의 평균을 구하시오

In [8]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).sum()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월01일,저평점,76000
1,11월02일,고평점,32000
2,11월02일,저평점,24000
3,11월03일,고평점,25000
4,11월03일,저평점,26000
5,11월04일,고평점,11000
6,11월04일,저평점,7000
7,11월05일,저평점,11000
8,11월06일,저평점,35000
9,11월07일,고평점,25000


In [9]:
# ... 평균을 구하시오
df_gb_rate.groupby(['날짜']).mean().astype(int)

,상품가격
날짜,
11월01일,76000
11월02일,28000
11월03일,25500
11월04일,9000
11월05일,11000
11월06일,35000
11월07일,48500
11월08일,85000
11월09일,63500


<a name="min"></a>
#### Min Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 최소값을 구한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 평균을 구하고 (Metric Aggregation : Average)
    * 그 평균(=상품가격들의 평균들의)들의 최소값을 구하시오

In [10]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).mean()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월01일,저평점,15200
1,11월02일,고평점,16000
2,11월02일,저평점,12000
3,11월03일,고평점,25000
4,11월03일,저평점,13000
5,11월04일,고평점,5500
6,11월04일,저평점,7000
7,11월05일,저평점,11000
8,11월06일,저평점,11666
9,11월07일,고평점,12500


In [11]:
# ... 최소값을 구하시오
df_gb_rate.groupby(['날짜']).min()[['상품가격']]

,상품가격
날짜,
11월01일,15200
11월02일,12000
11월03일,13000
11월04일,5500
11월05일,11000
11월06일,11666
11월07일,12500
11월08일,17000
11월09일,17333


<a name="max"></a>
#### Max Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 최대값을 구한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 평균을 구하고 (Metric Aggregation : Average)
    * 그 평균(=상품가격들의 평균들의)들의 최대값을 구하시오

In [12]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).mean()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월01일,저평점,15200
1,11월02일,고평점,16000
2,11월02일,저평점,12000
3,11월03일,고평점,25000
4,11월03일,저평점,13000
5,11월04일,고평점,5500
6,11월04일,저평점,7000
7,11월05일,저평점,11000
8,11월06일,저평점,11666
9,11월07일,고평점,12500


In [13]:
# ... 최대값을 구하시오
df_gb_rate.groupby(['날짜']).max()[['상품가격']]

,상품가격
날짜,
11월01일,15200
11월02일,16000
11월03일,25000
11월04일,7000
11월05일,11000
11월06일,11666
11월07일,18000
11월08일,17000
11월09일,23000
